In [1]:
import pandas as pd
from math import sqrt
import numpy as np

In [19]:
book_df=pd.read_csv("selected_books.csv", low_memory=False)
ratings_df=pd.read_csv('Ratings.csv', low_memory=False)

In [20]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Year-Of-Publication  271360 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [21]:
book_df['Book-Title'].value_counts()

Book-Title
Selected Poems                                                                27
Little Women                                                                  24
Wuthering Heights                                                             21
Dracula                                                                       20
Adventures of Huckleberry Finn                                                20
                                                                              ..
Rob Roy (Penguin Classics)                                                     1
Kids Who Laugh : How to Develop Your Child's Sense of Humor                    1
Sacred Self, Sacred Relationships: Healing the World for Seven Generations     1
Consenting Adults: Or, the Duchess Will Be Furious : A Novel                   1
A Little Princess (Penguin Classics)                                           1
Name: count, Length: 242135, dtype: int64

In [22]:
userInput = [
      
             {'Book-Title':'Decision in Normandy', 'Book-Rating':2},
             {'Book-Title':'Nights Below Station Street', 'Book-Rating':5},
             {'Book-Title':'Beloved (Plume Contemporary Fiction)', 'Book-Rating':5},
             {'Book-Title':'Goodbye to the Buttermilk Sky', 'Book-Rating':5},
             {'Book-Title':'Petite histoire de la dÃ?Â©sinformation', 'Book-Rating':4.3},
             {'Book-Title':'The Middle Stories', 'Book-Rating':6},
             {'Book-Title':'Flashpoints: Promise and Peril in a New World', 'Book-Rating':6.3},
             {'Book-Title':'The Witchfinder (Amos Walker Mystery Series)', 'Book-Rating':3},
             {'Book-Title':'Anti Death League', 'Book-Rating':4.5},]
inputBooks = pd.DataFrame(userInput)
inputBooks

,Book-Title,Book-Rating
0,Decision in Normandy,2.0
1,Nights Below Station Street,5.0
2,Beloved (Plume Contemporary Fiction),5.0
3,Goodbye to the Buttermilk Sky,5.0
4,Petite histoire de la dÃ?Â©sinformation,4.3
5,The Middle Stories,6.0
6,Flashpoints: Promise and Peril in a New World,6.3
7,The Witchfinder (Amos Walker Mystery Series),3.0
8,Anti Death League,4.5


In [23]:
inputId = book_df[book_df['Book-Title'].isin(inputBooks['Book-Title'].tolist())]
inputBooks = pd.merge(inputId, inputBooks)
inputBooks = inputBooks[['ISBN','Book-Title', 'Book-Rating']]
print(inputBooks)

         ISBN                                     Book-Title  Book-Rating
0  0060973129                           Decision in Normandy          2.0
1  0771074670                    Nights Below Station Street          5.0
2  0887841740                             The Middle Stories          6.0
3  1567407781   The Witchfinder (Amos Walker Mystery Series)          3.0
4  1881320189                  Goodbye to the Buttermilk Sky          5.0
5  0452264464           Beloved (Plume Contemporary Fiction)          5.0
6  2268032019        Petite histoire de la dÃ?Â©sinformation          4.3
7  014002803X                              Anti Death League          4.5
8  0449906736  Flashpoints: Promise and Peril in a New World          6.3


In [24]:
userSubset = ratings_df[ratings_df['ISBN'].isin(inputBooks['ISBN'].tolist())]
print(userSubset.groupby('ISBN').count())

            User-ID  Book-Rating
ISBN                            
0060973129        3            3
014002803X        1            1
0449906736        1            1
0452264464      180          180
0771074670        6            6
0887841740        2            2
1567407781        1            1
1881320189        3            3


In [25]:
userSubsetGroup = userSubset.groupby(['User-ID'])

def take_5_elem(x):
    return len(x[1])
    
userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:8])


[((8,),       User-ID        ISBN  Book-Rating
9564        8  0060973129            0
9573        8  0771074670            0
9575        8  0887841740            5
9578        8  1567407781            6
9580        8  1881320189            7), ((11676,),        User-ID        ISBN  Book-Rating
50879    11676  0452264464            8
54043    11676  0771074670            0), ((9,),       User-ID        ISBN  Book-Rating
9582        9  0452264464            6), ((2954,),        User-ID        ISBN  Book-Rating
16919     2954  0060973129            8), ((3373,),        User-ID        ISBN  Book-Rating
18928     3373  0452264464           10), ((4781,),        User-ID        ISBN  Book-Rating
22471     4781  0452264464            0), ((5555,),        User-ID        ISBN  Book-Rating
23951     5555  0452264464            9), ((6026,),        User-ID        ISBN  Book-Rating
25093     6026  0452264464            0)]


In [26]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:

    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='ISBN')
    inputBooks = inputBooks.sort_values(by='ISBN')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    temp_df = inputBooks[inputBooks['ISBN'].isin(group['ISBN'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['Book-Rating'].tolist()
   
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['Book-Rating'].tolist()
   
    
    #Now let's calculate the pearson correlation between two users, so called, x and y manually (check the formula from week 7 slide)
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0
    

In [27]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User-ID'] = [tup[0] for tup in pearsonDF.index]  # Extract the first element from each tuple
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()


,similarityIndex,User-ID
0,0.334927,8
1,0.000000,11676
2,0.000000,9
3,0.000000,2954
4,0.000000,3373


In [28]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  User-ID
0          0.334927        8
73         0.000000   102359
72         0.000000   102327
71         0.000000   101851
70         0.000000   101559


In [29]:
topUsersRating=topUsers.merge(ratings_df, left_on='User-ID', right_on='User-ID', how='inner')
print(topUsersRating.head(100))

    similarityIndex  User-ID        ISBN  Book-Rating
0          0.334927        8  0002005018            5
1          0.334927        8  0060973129            0
2          0.334927        8  0374157065            0
3          0.334927        8  0393045218            0
4          0.334927        8  0399135782            0
..              ...      ...         ...          ...
95         0.000000   102359  0670691992            9
96         0.000000   102359   067081302           10
97         0.000000   102359  0670813648           10
98         0.000000   102359  067081458X           10
99         0.000000   102359  067082982X           10

[100 rows x 4 columns]


In [30]:
#Multiplies the similarity by the user’s ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Book-Rating']
print(topUsersRating.head())

   similarityIndex  User-ID        ISBN  Book-Rating  weightedRating
0         0.334927        8  0002005018            5        1.674637
1         0.334927        8  0060973129            0        0.000000
2         0.334927        8  0374157065            0        0.000000
3         0.334927        8  0393045218            0        0.000000
4         0.334927        8  0399135782            0        0.000000


In [31]:
#Applies a sum to the topUsers after grouping it up by ISBN
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

            sum_similarityIndex  sum_weightedRating
ISBN                                               
0002000288             0.000000            0.000000
0002005018             0.334927            1.674637
0002172739             0.000000            0.000000
0002215497             0.000000            0.000000
0002219808             0.000000            0.000000


In [32]:
recommendation_df = pd.DataFrame()

# Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating'] / tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index

# Handle division by zero
recommendation_df['weighted average recommendation score'].replace([np.inf, -np.inf], np.nan, inplace=True)
recommendation_df['weighted average recommendation score'].fillna(0, inplace=True)

print(recommendation_df.head(10))


            weighted average recommendation score        ISBN
ISBN                                                         
0002000288                                    0.0  0002000288
0002005018                                    5.0  0002005018
0002172739                                    0.0  0002172739
0002215497                                    0.0  0002215497
0002219808                                    0.0  0002219808
0002224682                                    0.0  0002224682
0002244098                                    0.0  0002244098
000225056X                                    0.0  000225056X
0002255006                                    0.0  0002255006
0002255243                                    0.0  0002255243


In [33]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df.head())


            weighted average recommendation score        ISBN
ISBN                                                         
1881320189                                    7.0  1881320189
1575663937                                    6.0  1575663937
1567407781                                    6.0  1567407781
0887841740                                    5.0  0887841740
074322678X                                    5.0  074322678X


In [34]:
recommended_book=book_df.loc[book_df['ISBN'].isin(recommendation_df['ISBN'])]
recommended_book=recommended_book.loc[~recommended_book.ISBN.isin(userSubset['ISBN'])]
recommended_book.head()

,ISBN,Book-Title,Year-Of-Publication
1,0002005018,Clara Callan,2001
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,1999
4,0393045218,The Mummies of Urumchi,1999
5,0399135782,The Kitchen God's Wife,1991
6,0425176428,What If?: The World's Foremost Military Histor...,2000
